In [5]:
import warnings
warnings.filterwarnings("ignore")
from transformers import pipeline   #pipelines   
import torch

In [6]:
classifier = pipeline('sentiment-analysis')  #loading the sentiment analysis pipeline
classifier(
    ["I've been waiting for a HuggingFace course my whole life.",
      "I hate this so much!",
      ])  #testing the pipeline

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: Numpy is not available

In [1]:

from transformers import CLIPProcessor, CLIPModel
from PIL import Image

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Function to get text embeddings
def get_text_embedding(text):
    inputs = processor(text=text, return_tensors="pt", padding=True)
    text_embeddings = model.get_text_features(**inputs)
    return text_embeddings

# Function to get image embeddings
def get_image_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    image_embeddings = model.get_image_features(**inputs)
    return image_embeddings

# Example usage
text = "A photo of a cat"
image_path = "path_to_your_image.jpg"

text_embedding = get_text_embedding(text)
image_embedding = get_image_embedding(image_path)

# Calculate similarity
similarity = torch.nn.functional.cosine_similarity(text_embedding, image_embedding)
print(f"Similarity: {similarity.item()}")

ModuleNotFoundError: No module named 'torch'